In [1]:
import os
import sys
import csv
import pandas as pd
import numpy as np

In [2]:
cols = ['DIMENSION','ITERATION', 'NODE','CPU','PROCESS','THREAD', 'TIME']
serial_cols = ['DIMENSION','ITERATION', 'TYPE', 'TIME']

In [3]:
tests = []
all_infos_tests = []
serial_tests = []
for file in os.listdir("../logs",):
    r_index = file.rfind('.')
    l_index = file.rfind('/')
    fname = file[l_index+1:r_index]
    el = fname.split('_')[1:]
    el_complete = fname.split('_')[1:]
    if file.startswith('.'):
        continue
    
    with open("logs/"+file, 'r') as fp:
        if "serial" in file:
            try:
                line = fp.readline()
            except Exception as e:
                print(file)
                raise e
        else:
            try:
                line = fp.readline()
                el_complete.append(line.split(":")[1].replace("seconds", "").strip())
                line = fp.readline()
                el_complete.append(line.split(":")[1].replace("seconds", "").strip())
                line = fp.readline()
                el_complete.append(line.split(":")[1].replace("seconds", "").strip())
                line = fp.readline()
                el_complete.append(line.split(":")[1].replace("seconds", "").strip())
                line = fp.readline()
            except Exception as e:
                print(file)
                raise e
        try:
            if not el[0] in line:
                el.append(np.nan)
                el_complete.append(np.nan)
            else:
                el.append(line.split(',')[1])
                el_complete.append(line.split(',')[1])
        except Exception as e:
            print(file, fname, el)
            raise e
    if "serial" in el:
        serial_tests.append(el)
    else:
        tests.append(el)
        all_infos_tests.append(el_complete)


In [4]:
complete_cols = ['DIMENSION','ITERATION', 'NODE','CPU','PROCESS','THREAD', 'INITIALIZATION', 'DISTRIBUTION', 'CHUNKS', 'RECONSTRUCTION', 'TIME']
complete_df = pd.DataFrame(all_infos_tests, columns=complete_cols, )

In [5]:
complete_df

,DIMENSION,ITERATION,NODE,CPU,PROCESS,THREAD,INITIALIZATION,DISTRIBUTION,CHUNKS,RECONSTRUCTION,TIME
0,256,3,1,1,4,2,0.010000,0.020000,0.090000,0.000000,0.275595
1,4096,3,1,4,16,8,1.020000,1.090000,47.310000,0.080000,11576.875189
2,1024,3,2,4,16,8,0.080000,0.160000,0.680000,0.000000,62.222703
3,256,5,2,2,4,8,0.010000,0.110000,0.150000,0.000000,0.466758
4,256,4,2,4,16,4,0.040000,0.140000,0.230000,0.000000,0.369196
...,...,...,...,...,...,...,...,...,...,...,...
1671,1024,1,1,4,4,8,0.070000,0.420000,1.040000,0.010000,37.572280
1672,256,2,1,4,1,8,0.000000,0.060000,0.080000,0.010000,0.129274
1673,4096,3,1,8,4,1,1.230000,1.270000,247.140000,0.110000,19825.608467
1674,1024,3,1,1,16,1,0.070000,0.060000,1.440000,0.000000,38.542739


In [6]:
df = pd.DataFrame(tests, columns=cols, )

In [7]:
df = df.astype({'DIMENSION':int ,'ITERATION':int, 'NODE':int,'CPU':int,'PROCESS':int,'THREAD':int, 'TIME':float})
df.dtypes

DIMENSION      int64
ITERATION      int64
NODE           int64
CPU            int64
PROCESS        int64
THREAD         int64
TIME         float64
dtype: object

In [16]:
complete_df = complete_df.astype({'DIMENSION':int ,'ITERATION':int, 'NODE':int,'CPU':int,'PROCESS':int,'THREAD':int, 'INITIALIZATION':float, 'DISTRIBUTION':float, 'CHUNKS':float, 'RECONSTRUCTION':float, 'TIME':float})
complete_df.dtypes

DIMENSION           int64
ITERATION           int64
NODE                int64
CPU                 int64
PROCESS             int64
THREAD              int64
INITIALIZATION    float64
DISTRIBUTION      float64
CHUNKS            float64
RECONSTRUCTION    float64
TIME              float64
dtype: object

In [9]:
serial_df = pd.DataFrame(serial_tests, columns=serial_cols)
serial_df = serial_df.astype({'DIMENSION':int ,'ITERATION':int, 'TYPE':str, 'TIME':float})
serial_df.groupby(["DIMENSION"])["TIME"].mean().reset_index().sort_values(["DIMENSION"])

,DIMENSION,TIME
0,64,0.000
1,256,0.024
2,1024,0.186
3,4096,17.610


In [10]:
serial_df.groupby(['DIMENSION'])['TIME'].mean().reset_index()

,DIMENSION,TIME
0,64,0.000
1,256,0.024
2,1024,0.186
3,4096,17.610


In [11]:
res = df.groupby(['DIMENSION', 'NODE', 'CPU', 'PROCESS', 'THREAD'])['TIME'].mean().reset_index()

In [12]:
res

,DIMENSION,NODE,CPU,PROCESS,THREAD,TIME
0,64,1,1,1,1,0.007967
1,64,1,1,1,2,0.015584
2,64,1,1,1,4,0.014569
3,64,1,1,1,8,0.034349
4,64,1,1,4,1,0.013878
...,...,...,...,...,...,...
331,4096,2,8,4,8,23535.181490
332,4096,2,8,16,1,21541.890982
333,4096,2,8,16,2,18977.168052
334,4096,2,8,16,4,24301.382245


In [13]:
res.to_csv("results.csv", index=None)

In [17]:
complete_res = complete_df.groupby(['DIMENSION', 'NODE', 'CPU', 'PROCESS', 'THREAD'])['INITIALIZATION', 'DISTRIBUTION', 'CHUNKS', 'RECONSTRUCTION', 'TIME'].mean().reset_index()

/var/folders/s0/6q5821790kg2kpdsvjywc4xw0000gn/T/ipykernel_40324/224454142.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  complete_res = complete_df.groupby(['DIMENSION', 'NODE', 'CPU', 'PROCESS', 'THREAD'])['INITIALIZATION', 'DISTRIBUTION', 'CHUNKS', 'RECONSTRUCTION', 'TIME'].mean().reset_index()


In [20]:
complete_res

,DIMENSION,NODE,CPU,PROCESS,THREAD,INITIALIZATION,DISTRIBUTION,CHUNKS,RECONSTRUCTION,TIME
0,64,1,1,1,1,0.000,0.000,0.000,0.000,0.007967
1,64,1,1,1,2,0.000,0.000,0.002,0.000,0.015584
2,64,1,1,1,4,0.000,0.006,0.002,0.002,0.014569
3,64,1,1,1,8,0.000,0.038,0.038,0.002,0.034349
4,64,1,1,4,1,0.000,0.000,0.000,0.000,0.013878
...,...,...,...,...,...,...,...,...,...,...
331,4096,2,8,4,8,1.118,1.214,132.808,0.112,23535.181490
332,4096,2,8,16,1,1.250,1.162,119.312,0.096,21541.890982
333,4096,2,8,16,2,1.046,0.980,77.324,0.084,18977.168052
334,4096,2,8,16,4,1.196,1.240,55.792,0.098,24301.382245


In [33]:
df.to_csv("raw_results.csv", index=None)

In [21]:
complete_res.to_csv("complete_results.csv", index=None)